# Task1

## Preparing Data

In [2]:
!pwd
import os
os.getcwd() 
import sys
sys.path.append('/Users/zhouxudong/anaconda3/envs/ml/lib/python3.10/site-packages')

/home/flexiv/Desktop/Machine Learning/project


In [3]:
import torch

if torch.backends.mps.is_available():
    print("Metal is available!")
    device = torch.device("mps")
elif torch.cuda.is_available():
    print("Cuda is available!")
    device = torch.device("cuda")
else:
    print("Metal is not available.")
    device = torch.device("cpu")

Cuda is available!


In [4]:
import os
import pickle

# 定义数据集路径
dataset_path = './project_data'

# 加载.pkl文件内容
def load_pkl_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# 检查数据集中的.pkl文件
def check_pkl_files(dataset_path):
    pkl_files = [f for f in os.listdir(dataset_path) if f.endswith('.pkl')]
    return pkl_files

# 加载并查看第一个.pkl文件内容
def load_first_pkl_file(pkl_files, dataset_path):
    # first_pkl_path = os.path.join(dataset_path, pkl_files[2])
    # print(pkl_files[2])
    first_pkl_path = os.path.join(dataset_path, 'adder_2.pkl')
    data = load_pkl_file(first_pkl_path)
    return data

# 示例使用
pkl_files = check_pkl_files(dataset_path)
if pkl_files:
    first_pkl_data = load_first_pkl_file(pkl_files, dataset_path)
    print(first_pkl_data)
else:
    print("No .pkl files found in the dataset.")

{'input': ['adder_', 'adder_0', 'adder_03', 'adder_034', 'adder_0344', 'adder_03445', 'adder_034451', 'adder_0344515', 'adder_03445156', 'adder_034451563', 'adder_0344515636'], 'target': [0.13195968288377902, 0.1325812491538423, 0.0, 0.061099607830101293, 0.061099607830101293, 0.061099607830101293, 0.06493653730643018, 0.06493653730643018, 0.06493653730643018, 0.061099607830101293, 0.061099607830101293]}


In [5]:
import os

# Example usage
state = 'adder_0'
circuitName, actions = state.split('_')
circuitPath = './InitialAIG/train/' + circuitName + '.aig'
libFile = './lib/7nm/7nm.lib'
logFile = 'alu2.log'
nextState = state + '.aig'  # 当前AIG文件

synthesisOpToPosDic = {
    0: "refactor",
    1: "refactor -z",
    2: "rewrite",
    3: "rewrite -z",
    4: "resub",
    5: "resub -z",
    6: "balance"
}

action_cmd = ''
for action in actions:
    action_cmd += (synthesisOpToPosDic[int(action)] + '; ')

yosys_path = 'oss-cad-suite/bin/'

abcRunCmd = f"{yosys_path}yosys-abc -c \"read {circuitPath}; {action_cmd} read_lib {libFile}; write {nextState}; print_stats\" > {logFile}"
os.system(abcRunCmd)


0

In [6]:
import re
abcRunCmd = f"{yosys_path}yosys-abc -c \"read {circuitPath}; read_lib {libFile}; map; topo; stime\" > {logFile}"

os.system(abcRunCmd)
with open(logFile) as f:
    area_information = re.findall(r'[a-zA-Z0-9.]+', f.readlines()[-1])


eval = float(area_information[-9]) * float(area_information[-4])
print(f'Evaluation: {eval}')

Evaluation: 3386533.8845


In [7]:
nextBench = state +'.bench'
RESYN2_CMD = "balance; rewrite; refactor; balance; rewrite; rewrite -z; balance; refactor -z; rewrite -z; balance;"
abcRunCmd = f"{yosys_path}yosys-abc -c \"read {circuitPath}; {RESYN2_CMD} read_lib {libFile}; write {nextState}; write_bench -l {nextBench}; map; topo; stime\" > {logFile}"
os.system(abcRunCmd)

with open(logFile) as f:
    area_information = re.findall('[a-zA-Z0-9.]+', f.readlines()[-1])
baseline = float(area_information[-9]) * float(area_information[-4])
eval = 1 - eval / baseline
print(f'Evaluation: {eval}')

Evaluation: 0.13195968288377902


In [10]:
import abc_py
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
import networkx as nx

# 创建 AbcInterface 实例
_abc = abc_py.AbcInterface()
_abc.start()

if not os.path.exists(circuitPath):
    raise FileNotFoundError(f"circuitPath file '{circuitPath}' not found")

_abc.read(circuitPath)
data = {}
numNodes = _abc.numNodes()

# 初始化 data 字典
data['node_type'] = np.zeros(numNodes, dtype=int)
data['num_inverted_predecessors'] = np.zeros(numNodes, dtype=int)

edge_src_index = []
edge_target_index = []

# 遍历所有节点并填充数据结构
for nodeIdx in range(numNodes):
    aigNode = _abc.aigNode(nodeIdx)
    nodeType = aigNode.nodeType()

    data['num_inverted_predecessors'][nodeIdx] = 0

    if nodeType == 0 or nodeType == 2:
        data['node_type'][nodeIdx] = 0
    elif nodeType == 1:
        data['node_type'][nodeIdx] = 1
    else:
        data['node_type'][nodeIdx] = 2

    if nodeType == 4:
        data['num_inverted_predecessors'][nodeIdx] = 1
    if nodeType == 5:
        data['num_inverted_predecessors'][nodeIdx] = 2

    if aigNode.hasFanin0():
        fanin = aigNode.fanin0()
        edge_src_index.append(nodeIdx)
        edge_target_index.append(fanin)

    if aigNode.hasFanin1():
        fanin = aigNode.fanin1()
        edge_src_index.append(nodeIdx)
        edge_target_index.append(fanin)

# 检查 edge 索引长度
if len(edge_src_index) != len(edge_target_index):
    raise ValueError("Mismatch in edge indices length")

data['edge_index'] = torch.tensor([edge_src_index, edge_target_index], dtype=torch.long)
data['node_type'] = torch.tensor(data['node_type'])
data['num_inverted_predecessors'] = torch.tensor(data['num_inverted_predecessors'])
data['nodes'] = numNodes

print("Data prepared successfully:", data)

# 可视化节点类型
plt.figure(figsize=(10, 6))
plt.hist(data['node_type'].numpy(), bins=np.arange(4) - 0.5, rwidth=0.8)
plt.xticks(ticks=[0, 1, 2], labels=['AND', 'PI', 'Other'])
plt.xlabel('Node Type')
plt.ylabel('Count')
plt.title('Distribution of Node Types')
plt.show()

# 可视化边
G = nx.DiGraph()
for i in range(numNodes):
    G.add_node(i, type=data['node_type'][i].item())

edges = list(zip(edge_src_index, edge_target_index))
G.add_edges_from(edges)

pos = nx.spring_layout(G)
node_colors = ['red' if G.nodes[i]['type'] == 0 else 'blue' if G.nodes[i]['type'] == 1 else 'green' for i in G.nodes]

plt.figure(figsize=(12, 8))
nx.draw(G, pos, node_color=node_colors, with_labels=True, node_size=500, font_size=10, font_color='white', arrows=True)
plt.title('Graph Visualization of AIG Nodes and Edges')
plt.show()


Help on module abc_py:

NAME
    abc_py

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        AbcInterface
        AigNode
        AigStats
    
    class AbcInterface(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      AbcInterface
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(...)
     |      __init__(self: abc_py.AbcInterface) -> None
     |  
     |  aigNode(...)
     |      aigNode(self: abc_py.AbcInterface, arg0: int) -> ABC_PY::AigNode
     |      
     |      Get one AigNode
     |  
     |  aigStats(...)
     |      aigStats(self: abc_py.AbcInterface) -> ABC_PY::AigStats
     |      
     |      Get the AIG stats from the ABC framework`
     |  
     |  balance(...)
     |      balance(self: abc_py.AbcInterface, l: bool = False, d: bool = False, s: bool = False, x: bool = False) -> bool
     |      
     |      balance action
     |  
     

FileNotFoundError: State file 'path_to_your_state_file.aig' not found

## Training

### Model Definition

In [8]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

class GNNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GNNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x, adj):
        h = torch.matmul(adj, x)
        h = self.linear(h)
        return h

class GNN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(GNN, self).__init__()
        self.gnn1 = GNNLayer(in_features, hidden_features)
        self.gnn2 = GNNLayer(hidden_features, out_features)

    def forward(self, x, adj):
        x = self.gnn1(x, adj)
        x = F.relu(x)
        x = self.gnn2(x, adj)
        return x

In [4]:
import os
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import abc_py

# 定义数据集路径
dataset_path = 'small/project_data/'
aig_path = 'small/InitialAIG/train'

# 使用 Yosys 工具解析 AIG 文件
def read_aig_file(yosys_path, aig_file_path):
    aig_data = {
        'nodes': [],
        'edges': []
    }
    yosys_cmd = f"{yosys_path} -p 'read_aiger {aig_file_path}; write_blif temp.blif' > /dev/null 2>&1"
    os.system(yosys_cmd)
    
    with open('temp.blif', 'r') as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith('.inputs'):
                inputs = line.split()[1:]
                for i, inp in enumerate(inputs):
                    aig_data['nodes'].append({'id': i, 'label': 'input'})
            elif line.startswith('.outputs'):
                outputs = line.split()[1:]
                for i, out in enumerate(outputs, start=len(aig_data['nodes'])):
                    aig_data['nodes'].append({'id': i, 'label': 'output'})
            elif line.startswith('.names'):
                parts = line.split()
                if len(parts) == 4:
                    src1 = parts[1]
                    src2 = parts[2]
                    target = parts[3]
                    src1_id = next((node['id'] for node in aig_data['nodes'] if node['label'] == src1), len(aig_data['nodes']))
                    src2_id = next((node['id'] for node in aig_data['nodes'] if node['label'] == src2), len(aig_data['nodes']))
                    target_id = next((node['id'] for node in aig_data['nodes'] if node['label'] == target), len(aig_data['nodes']))
                    aig_data['nodes'].append({'id': target_id, 'label': 'AND'})
                    aig_data['edges'].append({'source': src1_id, 'target': target_id})
                    aig_data['edges'].append({'source': src2_id, 'target': target_id})
    print(aig_data)
    return aig_data

# 加载数据
def load_data(dataset_path):
    data_list = []
    labels_list = []
    pkl_files = [f for f in os.listdir(dataset_path) if f.endswith('.pkl')]
    for pkl_file in pkl_files:
        with open(os.path.join(dataset_path, pkl_file), 'rb') as f:
            dataset = pickle.load(f)
            input_state = dataset['input']
            target_value = dataset['target']
            base_name = pkl_file.split('_')[0]
            aig_file = base_name + '.aig'
            aig_file_path = os.path.join(aig_path, aig_file)
            aig_data = read_aig_file(yosys_path, aig_file_path)
            data_list.append((input_state, aig_data))
            labels_list.append(target_value)
    print(data_list)
    print(labels_list)
    return data_list, labels_list

# 将输入状态转换为图数据
def build_graph_data(input_states, target_value, aig_data):
    node_features = np.array([[len(state)] for state in input_states], dtype=np.float32)
    edges = [(edge['source'], edge['target']) for edge in aig_data['edges']]
    edges = np.array(edges, dtype=np.int64).T
    target = np.array(target_value, dtype=np.float32)
    return node_features, edges, target

# 示例路径
yosys_path = 'oss-cad-suite/bin/yosys'

# 加载数据并构建图数据
print('loading...')
data_list, labels_list = load_data(dataset_path)
graph_data_list = [build_graph_data(input_states, target_value, aig_data) for (input_states, aig_data), target_value in zip(data_list, labels_list)]

# 创建自定义数据集
class GraphDataset(Dataset):
    def __init__(self, graph_data_list):
        self.graph_data_list = graph_data_list

    def __len__(self):
        return len(self.graph_data_list)

    def __getitem__(self, idx):
        return self.graph_data_list[idx]

# 划分训练集和测试集
train_data, test_data = train_test_split(graph_data_list, test_size=0.2, random_state=42)
train_dataset = GraphDataset(train_data)
test_dataset = GraphDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# 训练模型
def train(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for node_features, edges, targets in tqdm(train_loader, desc="Training"):  # 添加进度条
        print("Node features shape:", node_features.shape)
        print("Edges shape:", edges.shape)
        node_features = node_features.squeeze(0)  # 调整节点特征的形状
        edges = edges.squeeze(0)  # 调整边的形状
        optimizer.zero_grad()
        adj = torch.zeros((node_features.size(0), node_features.size(0)))  # 使用节点数量创建邻接矩阵
        if edges.shape[0] == 2:
            adj[edges[0], edges[1]] = 1
        adj += torch.eye(node_features.size(0))  # Add self-connections
        output = model(node_features, adj)
        loss = criterion(output.squeeze(), targets.squeeze(0))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# 测试模型
def test(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for node_features, edges, targets in tqdm(test_loader, desc="Testing"):  # 添加进度条
            node_features = node_features.squeeze(0)  # 调整节点特征的形状
            edges = edges.squeeze(0)  # 调整边的形状
            adj = torch.zeros((node_features.size(0), node_features.size(0)))  # 使用节点数量创建邻接矩阵
            if edges.shape[0] == 2:
                adj[edges[0], edges[1]] = 1
            adj += torch.eye(node_features.size(0))  # Add self-connections
            output = model(node_features, adj)
            loss = criterion(output.squeeze(), targets.squeeze(0))
            total_loss += loss.item()
    return total_loss / len(test_loader)

# 初始化模型、优化器和损失函数
model = GNN(in_features=1, hidden_features=16, out_features=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# 训练和测试模型
num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    if epoch % 10 == 0:
        test_loss = test(model, test_loader, criterion)
        print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# 测试模型性能
model.eval()
predictions = []
actuals = []
with torch.no_grad():
    for node_features, edges, targets in test_loader:
        node_features = node_features.squeeze(0)  # 调整节点特征的形状
        edges = edges.squeeze(0)  # 调整边的形状
        adj = torch.zeros((node_features.size(0), node_features.size(0)))  # 使用节点数量创建邻接矩阵
        if edges.shape[0] == 2:
            adj[edges[0], edges[1]] = 1
        adj += torch.eye(node_features.size(0))  # Add self-connections
        output = model(node_features, adj)
        predictions.extend(output.squeeze().tolist())
        actuals.extend(targets.squeeze(0).tolist())
mse = mean_squared_error(actuals, predictions)
print(f'Mean Squared Error on Test Set: {mse:.4f}')

loading...
{'nodes': [{'id': 0, 'label': 'input'}, {'id': 1, 'label': 'input'}, {'id': 2, 'label': 'input'}, {'id': 3, 'label': 'input'}, {'id': 4, 'label': 'input'}, {'id': 5, 'label': 'input'}, {'id': 6, 'label': 'input'}, {'id': 7, 'label': 'input'}, {'id': 8, 'label': 'input'}, {'id': 9, 'label': 'input'}, {'id': 10, 'label': 'input'}, {'id': 11, 'label': 'input'}, {'id': 12, 'label': 'input'}, {'id': 13, 'label': 'input'}, {'id': 14, 'label': 'input'}, {'id': 15, 'label': 'input'}, {'id': 16, 'label': 'input'}, {'id': 17, 'label': 'input'}, {'id': 18, 'label': 'input'}, {'id': 19, 'label': 'input'}, {'id': 20, 'label': 'input'}, {'id': 21, 'label': 'input'}, {'id': 22, 'label': 'input'}, {'id': 23, 'label': 'input'}, {'id': 24, 'label': 'input'}, {'id': 25, 'label': 'input'}, {'id': 26, 'label': 'input'}, {'id': 27, 'label': 'input'}, {'id': 28, 'label': 'input'}, {'id': 29, 'label': 'input'}, {'id': 30, 'label': 'input'}, {'id': 31, 'label': 'input'}, {'id': 32, 'label': 'input'}

Training:   0%|          | 0/8 [00:00<?, ?it/s]

Node features shape: torch.Size([1, 11, 1])
Edges shape: torch.Size([1, 2, 2040])


IndexError: index 385 is out of bounds for dimension 0 with size 11